# Use Cloud Pak for Data and Netezza Performance Server to Build a Churn Predictive Model

Cloud Pak for Data supports the entire Ladder to AI, including Collect, Organize, Analyze, and Infuse. Cloud Pak for Data enables companies to rapidly Modernize their applications on the Cloud of their choice, or on premise.

Netezza Performance Server for PostgreSQL (NPS) is an add-on data warehouse solution available on Cloud Pak for Data System platform, built over open source and optimized for High Performance Analytics with built-in hardware acceleration.


## Learning goals

You will learn how to:

-  Load data from Netezza Performance Server into a pandas dataframe
-  Explore data
-  Prepare data for training and evaluation
-  Create an XGBoost machine learning model
-  Train and evaluate the model
-  Use cross-validation to optimize model's hyperparameters
-  Persist the model in a Cloud Pak for Data deployment space
-  Deploy the model for online scoring
-  Score test data


## Contents

This notebook contains the following parts:

1.	[Set up the environment](#setup)
2.	[Load and explore the data](#load)
3.	[Create an XGBoost model](#model)
4.	[Deploy and score the model in Cloud Pak for Data](#scoring)
5.	[Summary and next steps](#summary)

<a id='setup'></a>
## 1. Set up the environment

Install and import packages as needed.

In [ ]:
!pip install --upgrade seaborn
import seaborn as sns
import matplotlib.pyplot as plt

!pip install scikit-learn==0.19.1
!pip install scikit-learn==0.20
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

!pip install 'xgboost==0.80'
import xgboost
xgboost.__version__
from xgboost.sklearn import XGBClassifier

<a id='load'></a>
## 2. Load and explore the data

In this section, you will load the data into a pandas dataframe and perform an exploratory data analysis.

In [ ]:
# Load data from NPS Performance Server

# This code is auto generated from "0100" Icon on Upper Right > Connections > InsertToCode

# This cell loads the Table: CUSTOMER_CHURN
# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.

from project_lib import Project
project = Project.access()
NPS_Database_credentials = project.get_connection(name="NPS_Database")

import jaydebeapi, pandas as pd
NPS_Database_connection = jaydebeapi.connect('org.netezza.Driver',
    '{}://{}:{}/{}'.format('jdbc:netezza',
    NPS_Database_credentials['host'],
    NPS_Database_credentials['port'],
    NPS_Database_credentials['database']),
    [NPS_Database_credentials['username'],
    NPS_Database_credentials['password']])

query = 'SELECT * FROM ADMIN.CUSTOMER_CHURN'
data_df_1 = pd.read_sql(query, con=NPS_Database_connection)
data_df_1.head()

# You can close the database connection with the following code.
# NPS_Database_connection.close()
# To learn more about the jaydebeapi package, please read the documentation: https://pypi.org/project/JayDeBeApi/


In [ ]:
# Load data from NPS Performance Server

# This code is auto generated from "0100" Icon on Upper Right > Connections > InsertToCode

# This cell loads the Table: CUSTOMER_DEMOGRAPHICS
query = 'SELECT * FROM ADMIN.CUSTOMER_DEMOGRAPHICS'
data_df_2 = pd.read_sql(query, con=NPS_Database_connection)
data_df_2.head()


In [ ]:
# Load data from NPS Performance Server

# This code is auto generated from "0100" Icon on Upper Right > Connections > InsertToCode

# This cell loads the Table: CUSTOMER_ACTIVITY
query = 'SELECT * FROM ADMIN.CUSTOMER_ACTIVITY'
data_df_3 = pd.read_sql(query, con=NPS_Database_connection)
data_df_3.head()


In [ ]:
# Join the three tables using an inner join on the key field: ID

df_4 = pd.merge(left=data_df_1, right=data_df_2, left_on='ID', right_on='ID')
df = pd.merge(left=df_4, right=data_df_3, left_on='ID', right_on='ID')
df.head()


Run the code in the next cell to view the predictor names and data types.

You can see that the data set has 569 data points and 31 predictors.

In [ ]:
# Information about the data set, predictor names, and data types.
df.info()

In [ ]:
# Information about values in the numerical columns.
df.describe()

You can see the distribution of the target values/labels by running the following code.

In [ ]:
# Distribution of target values/labels.
df['CHURNRISK'].value_counts()

In [ ]:
# Check for NANs.
df.isnull().sum().sum()

In order to make accurate predictions, you need to select the significant predictors by choosing the features that most affect the target: CHURNRISK.

In [ ]:
# Plot a correlation heatmap
plt.subplots(figsize=(25,20))
hm1 = sns.heatmap(df.corr(), annot=True, cmap='YlGnBu')
hm1.set_xticklabels(hm1.get_xticklabels(), rotation=90)
hm1.xaxis.set_ticks_position('top')

This correlation heatmap helps with feature selection because the color gradient shows the correlation between the columns of the dataframe. In order to select only the *significant* predictors, you must eliminate features that are highly correlated with each other **(ex: > 0.95)**.

With respect to predicting the labels, the most significant predictors can be found by plotting boxplots of the numerical values against the labels. The features with boxplots that show the most variance should be chosen as the predictors for your model.

In [ ]:
# plot boxplots of numerical columns
cont_list = ['CHILDREN', 'ESTINCOME', 'AGE', 'TOTALDALLARVALUETRADED', 'TOTALUNITSTRADED', 'LARGESTSINGLETRANSACTION', 'SMALLESTSINGLETRANSACTION', 'PERCENTCHANGECALCULATION', 'DAYSSINCELASTLOGIN', 'DAYSSINCELASTTRADE', 'NETREALIZEDGAINS_YTD', 'NETREALIZEDLOSSES_YTD']
#cont_list = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'symmetry_mean', 'fractal_dimension_mean']
f, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9), (ax10, ax11, ax12)) = plt.subplots(4, 3, figsize=(20, 25))
ax = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12]

for i in range(len(cont_list)):
    sns.boxplot(x = 'CHURNRISK', y = cont_list[i], data=df, ax=ax[i])

<a id='model'></a>
## 3. Create an XGBoost model

In this section, you will learn how to train and test an XGBoost model.

- [3.1 Split data](#prepare)
- [3.2 Create an XGBoost model](#create)

By plotting the boxplots of each numerical column against the diagnosis type, we have picked out the significant features/predictors. More variation in the boxplot implies higher significance. We also eliminate features that are highly correlated. Therefore we can choose *radius_mean, radius_se, compactness_worst, concavity_mean, texture_mean* as the predictors for our model.

### 3.1 Split data<a id='prepare'></a>

You will pass the data with the selected significant predictors to build the model. You will use the `diagnosis` column as your target variable.

In [ ]:
df.head()

In [ ]:
# Choosing the inputs to the model

# Define input data to the model
X = df.drop(['GENDER','HOMEOWNER','STATUS','ID','CHURNRISK','TAXID','CREDITCARD','DOB','ADDRESS_1', 'ADDRESS_2', 'CITY', 'STATE', 'ZIP', 'ZIP4', 'LONGITUDE',
       'LATITUDE'], axis=1)
X = X.values

# Changing the some variables to binary variables
y = (df['CHURNRISK'] == 'High').astype(int)  # Converting from High, Medium, Low, to binary values representing High and Not-High
y = y.values

Split the data set into: 
- Train data set
- Test data set

In [ ]:
# Split the data set and create two data sets.
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=143)

In [ ]:
# List the number of records in each data set.
print('Number of training records: ' + str(len(X_train)))
print('Number of testing records : ' + str(len(X_test)))

The data has been successfully split into two data sets:
- The train data set which is the largest group will be used for training.
- The test data set will be used for model evaluation and is used to test the assumptions of the model.

### 3.2 Create an XGBoost model<a id='create'></a>

#### 3.2.1 Create an XGBoost classifier

In this subsection, you will create an XGBoost classifier with default hyperparameters and you will call *xgb_model*. 

**Note**: The next sections show you how to improve this base model.

In this first building of the model will use 100 iterations. Then we'll examine how accuracy varies over the iterations. Finally, we'll rebuild the model using the number of iterations that leads to and accurate, but not overfit, model.

In [ ]:
# Create the XGB classifier - xgb_model.
xgb_model = XGBClassifier(n_estimators=100)

Display the default parameters for *xgb_model*.

In [ ]:
# List the default parameters.
print(xgb_model.get_xgb_params())

Now, that your XGBoost classifier *xgb_model* is set up, you can train it by using the fit method. You will also evaluate *xgb_model* as the train and test data are being trained.

In [ ]:
# Train and evaluate.
xgb_model.fit(X_train, y_train, eval_metric=['error'], eval_set=[((X_train, y_train)),(X_test, y_test)])

Plot the model performance evaluated during the training process to assess model overfitting.

In [ ]:
# Plot and display the performance evaluation
xgb_eval = xgb_model.evals_result()
eval_steps = range(len(xgb_eval['validation_0']['error']))

fig, ax = plt.subplots(1, 1, sharex=True, figsize=(8, 6))

ax.plot(eval_steps, [1-x for x in xgb_eval['validation_0']['error']], label='Train')
ax.plot(eval_steps, [1-x for x in xgb_eval['validation_1']['error']], label='Test')
ax.legend()
ax.set_title('Accuracy')
ax.set_xlabel('Number of iterations')

You can see that there is model overfitting, and there is no increase in model accuracy after about 35 iterations.

So let's rebuild the model using 35 iterations.

In [ ]:
xgb_model = XGBClassifier(n_estimators=35)
xgb_model.fit(X_train, y_train, eval_metric=['error'], eval_set=[((X_train, y_train)),(X_test, y_test)])

Again, let's plot the model performance evaluated during the training process to assess model overfitting. This looks alot better.

In [ ]:
# Plot and display the performance evaluation
xgb_eval = xgb_model.evals_result()
eval_steps = range(len(xgb_eval['validation_0']['error']))

fig, ax = plt.subplots(1, 1, sharex=True, figsize=(8, 6))

ax.plot(eval_steps, [1-x for x in xgb_eval['validation_0']['error']], label='Train')
ax.plot(eval_steps, [1-x for x in xgb_eval['validation_1']['error']], label='Test')
ax.legend()
ax.set_title('Accuracy')
ax.set_xlabel('Number of iterations')

In [ ]:
# Select trained model.
n_trees = 35
y_pred = xgb_model.predict(X_test, ntree_limit= n_trees)

In [ ]:
# Check the accuracy of the trained model.
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f%%' % (accuracy * 100.0))

**Note:** You will use the test data accuracy to compare the accuracy of the model with *default* parameters to the accuracy of the model with *tuned* parameters.

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
plt.matshow(cm)
plt.colorbar()
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.show()
pd.DataFrame(cm)

This confusion matrix maps the predicted values against the actual values. Here, you can see that 126 benign tumors and 66 malignant tumors have been predicted correctly. However, 8 benign tumors have been incorrectly predicted as malignant. 

In [ ]:
y_pred_prob = xgb_model.predict_proba(X_test)

# ROC-AUC curve
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:, 1])
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

This is the ROC-AUC curve - the area under the curve represents the accuracy of the predictions. You can see that the area under the curve is large, indicating that the predictions are highly accurate.

#### 3.2.2 Use grid search and cross-validation to tune the model 

You can use grid search and cross-validation to tune your model to achieve better accuracy.

**Note**: Grid search is used for this model as an example, but it is **not** recommended for small data sets such as this one, as it might lead to overfitting.

XGBoost has an extensive catalog of hyperparameters which provides great flexibility to shape an algorithm’s desired behavior. Here you will the optimize the model tuning which adds an L1 penalty (`reg_alpha`).

Use a 5-fold cross-validation because your training data set is small.

In the cell below, create the XGBoost pipeline and set up the parameter grid for the grid search.

In [ ]:
# Create XGBoost pipeline, set up parameter grid.
xgb_model_gs = XGBClassifier()
parameters = {'reg_alpha': [0.0, 1.0, 2.0], 'reg_lambda': [0.0, 1.0, 2.0], 'n_estimators': [n_trees], 'seed': [1337]}

Use ``GridSearchCV`` to search for the best parameters from the specified values in the previous cell.

In [ ]:
# Search for the best parameters.
clf = GridSearchCV(estimator = xgb_model_gs, param_grid = parameters, scoring='accuracy', cv=5, verbose=1, n_jobs=1, refit=True)
clf.fit(X_train, y_train)

You can see the cross validation results that were evaluated by the grid search.

In [ ]:
# Print model cross validation results.
for key in ['params', 'mean_train_score', 'std_train_score', 'mean_test_score', 'std_test_score']:
    print(str(key) + ': \n' + str(clf.cv_results_[key]) + '\n\n')

Display the accuracy estimated using cross-validation and the hyperparameter values for the best model.

In [ ]:
print('Best score: %.1f%%' % (clf.best_score_*100))
print('Best parameter set: %s' % (clf.best_params_))

Display the accuracy of the best parameter combination on the test set.

In [ ]:
y_pred = clf.best_estimator_.predict(X_test, ntree_limit= n_trees)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f%%' % (accuracy * 100.0))

The test set's accuracy is about the same for both the tuned model and the trained model with default hyperparameter values, even though the tuned hyperparameters are different from the default parameters.

# 4. Deploy and Score the Model In Cloud Pak for Data <a id="scoring"></a>


In this section, you will learn how to deploy and use models in Cloud Pak for Data.

Before we save the model we must create a deployment space. Cloud Pak for Data provides deployment spaces where the user can save, configure and deploy their models. We can save models, functions and data assets in this space.

The steps involved for saving and deploying the model are as follows:

1. Create a new deployment space. Enter the name of the space in the cell below. Specify a tag for the space in the code cell below. This tag will be used in the future to identify this space. If a space with specified space_name already exists, existing space will be deleted before creating a new space.
2. Set this deployment space as the default space.
3. Store the model pipeline in the deployment space. Enter the name for the model in the cell below. Specify a tag for the model in the cell below.
4. Deploy the saved model. Enter the deployment name in the cell below. Specifu a tag for the deployment. Similarily, this tag will be used in the future to identify this deployment.
5. Retrieve the scoring endpoint to score the model with a payload
5. We will use the watson_machine_learning_client package to complete these steps. 

In [ ]:
# Specify a names for the space being created, the saved model and the model deployment
space_name = 'Deployment-Space-Created-From-Notebook'
space_tag = 'Deployment-Space-Created-From-Notebook-tag'

model_name = 'Churn-Model'
model_tag = 'Churn-Model-tag'

deployment_name = 'Churn-Model-Deployment'
deployment_tag = 'Churn-Model-Deployment-Tag'

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.0.0"
}

client = WatsonMachineLearningAPIClient(wml_credentials)

If a space with specified space_name already exists, delete the existing space before creating a new one.


In [ ]:
for space in client.spaces.get_details()['resources']:
    if space_name in space['entity']['name']:
        client.spaces.delete(space['metadata']['guid'])
        print(space_name, "is deleted")

### 4.1 Create Deployment Space

In [ ]:
# create the space and set it as default
space_meta_data = {
        client.spaces.ConfigurationMetaNames.NAME : space_name,
        client.spaces.ConfigurationMetaNames.TAGS : [{'value': space_tag}]
}

stored_space_details = client.spaces.store(space_meta_data)

space_uid = stored_space_details['metadata']['guid']

# set the newly created deployment space as the default
client.set.default_space(space_uid)

In [ ]:
# fetching details of the space created
stored_space_details

### 4.2 Store the model in the deployment space

In [ ]:
# run this line if you do not know the version of scikit-learn that was used to build the model
!pip list | grep scikit-learn

In [ ]:
metadata = {
    client.repository.ModelMetaNames.NAME: model_name,
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20",
    client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3",
    client.repository.ModelMetaNames.TAGS: [{'value' : model_tag}],
    client.repository.ModelMetaNames.SPACE_UID: space_uid
}

stored_model_details = client.repository.store_model(xgb_model,
                                               meta_props=metadata,
                                               training_data=X_train,
                                               training_target=y_train)

In [ ]:
stored_model_details

### 4.3 Create a deployment for the stored model

In [ ]:
# deploy the model for on-line use
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: deployment_name,
    client.deployments.ConfigurationMetaNames.TAGS : [{'value' : deployment_tag}],
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the model

model_uid = stored_model_details["metadata"]["guid"]
deployment_details = client.deployments.create( artifact_uid=model_uid, meta_props=meta_props)
deployment_details

### 4.4 Score Using the Deployed, On-line Model

In [ ]:
# retrieve the scoring endpoint
scoring_endpoint = client.deployments.get_scoring_href(deployment_details)

print('Scoring Endpoint:   ',scoring_endpoint)

In [ ]:
scoring_deployment_id = client.deployments.get_uid(deployment_details)
client.deployments.get_details(scoring_deployment_id)

In [ ]:
payload = [{"values": [ [1, 38000, 24, 1200000, 509, 9400, 940, 51, 3, 10, 0, -81000]]}]

In [ ]:
payload_metadata = {client.deployments.ScoringMetaNames.INPUT_DATA: payload}
# score
predictions = client.deployments.score(scoring_deployment_id, payload_metadata)
predictions

<a id='summary'></a>
## 5. Conclusion     

You have successfully completed this notebook! 

You learned how to use Cloud Pak for Data with Netezza Performance Server. 

### Authors

Tom Konchan, a Data Scientist at IBM derived this notebook from a notebook found in the Watson Studio Gallery: [Use XGBoost to classify tumors](https://dataplatform.cloud.ibm.com/exchange/public/entry/view/30b8df0ef74241a0516f9e81cd6d7029). It was originally developed by:

**Wojciech Sobala**, a Data Scientist at IBM.  
**Ananya Kaushik**, a Data Scientist at IBM.

Copyright © 2017-2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style='background:#F5F7FA; height:110px; padding: 2em; font-size:14px;'>
<span style='font-size:18px;color:#152935;'>Love this notebook? </span>
<span style='font-size:15px;color:#152935;float:right;margin-right:40px;'>Don't have an account yet?</span><br>
<span style='color:#5A6872;'>Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style='border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;'><a href='https://ibm.co/wsnotebooks' target='_blank' style='color: #3d70b2;text-decoration: none;'>Sign Up</a></span><br>
</div>